In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
DATA = "data.csv"
data = pd.read_csv(DATA)
data = data.dropna()
data.head()

,product_id,product_title,category,product_subcategory,brand,selling_price,original_price,product_type,product_rating,product_description,availability_status,customer_reviews_count,seasonal_indicator,promotion_indicator,shipping_weight,bundle_indicator,customer_demographics
0,1,Garlic Oil - Vegetarian Capsule 500 mg,Beauty & Hygiene,Hair Care,Sri Sri Ayurveda,220.0,220.0,Hair Oil & Serum,4.1,This Product contains Garlic Oil that is known...,In Stock,7,Spring,Yes,0.658145,Individual,Female
1,2,Water Bottle - Orange,"Kitchen, Garden & Pets",Storage & Accessories,Mastercook,180.0,180.0,Water & Fridge Bottles,2.3,"Each product is microwave safe (without lid), ...",In Stock,54,Winter,No,4.734585,Bundle,Female
2,3,"Brass Angle Deep - Plain, No.2",Cleaning & Household,Pooja Needs,Trm,119.0,250.0,Lamp & Lamp Oil,3.4,"A perfect gift for all occasions, be it your m...",In Stock,31,Winter,No,2.024140,Bundle,Female
3,4,Cereal Flip Lid Container/Storage Jar - Assort...,Cleaning & Household,Bins & Bathroom Ware,Nakoda,149.0,176.0,"Laundry, Storage Baskets",3.7,Multipurpose container with an attractive desi...,In Stock,49,Spring,No,3.869239,Individual,Other
4,5,Creme Soft Soap - For Hands & Body,Beauty & Hygiene,Bath & Hand Wash,Nivea,162.0,162.0,Bathing Bars & Soaps,4.4,Nivea Creme Soft Soap gives your skin the best...,In Stock,74,Winter,Yes,0.812489,Individual,Male


In [9]:
# D = data.where(data['product_subcategory'] == 'Hair Care').groupby('brand').agg(product_type_count=('product_type', 'count'), product_rating_mean=('product_rating', 'mean')).sort_values('product_type_count', ascending=False).sort_values('product_rating_mean', ascending=False)

In [10]:
# D = data.groupby('brand').agg(product_subcategory=('product_subcategory', 'first'), product_type_count=('product_type', 'count'), product_rating_mean=('product_rating', 'mean')).sort_values('product_type_count', ascending=False).sort_values('product_rating_mean', ascending=False)
# D = D[D.product_type_count > 5]

In [11]:
# hair_care_products = D[D['product_subcategory'] == 'Hair Care']
# num_bins = 5

# hair_care_products['rating_bin'] = pd.qcut(hair_care_products['product_type_count'], q=num_bins, labels=[f"{i+1}" for i in range(num_bins)])
# hair_care_products['rating_bin'] = hair_care_products['rating_bin'].astype(int)
# print(hair_care_products.groupby('rating_bin').agg(le=('rating_bin', 'count')))

In [12]:
# hair_care_products.to_json('hair_care_products.json', orient='records')

In [47]:
import numpy as np
import pandas as pd

# Assume 'data' is your DataFrame, filled with relevant data

def get_heatmap(subcategory):
    # Take Product Subcategory
    print(subcategory)
    # Group by Brand
    D = data.groupby('brand').agg(
        product_subcategory=('product_subcategory', 'first'), 
        product_type_count=('product_type', 'count'), 
        product_rating_mean=('product_rating', 'mean')
    )

    # Extract Brands with more than 5 products
    D = D[D.product_type_count > 5]

    # Assign Bins
    num_bins = 5
    DS = D[D['product_subcategory'] == subcategory].copy()
    if DS.empty:
        return
    DS['rating_bin'] = pd.qcut(DS['product_type_count'], q=num_bins, labels=False, duplicates='drop') + 1  # Labels=False for easier manipulation

    # Check the distribution of counts per bin
    bin_counts = DS['rating_bin'].value_counts()
    max_length = bin_counts.max()

    # Increment the num_bins and do qcut if the max length of bin is greater than 7
    while max_length > 7 and num_bins < 9:
        num_bins += 1
        DS['rating_bin'] = pd.qcut(DS['product_type_count'], q=num_bins, labels=False, duplicates='drop') + 1
        bin_counts = DS['rating_bin'].value_counts()
        max_length = bin_counts.max()

    # After final binning, assign proper labels
    DS['rating_bin'] = pd.Categorical(DS['rating_bin'], categories=np.arange(1, num_bins+1))
    print(num_bins)

    # In a Bin, sort the brands on the basis of rating, and assign serial numbers to each brand
    DS = DS.sort_values(['rating_bin', 'product_rating_mean'], ascending=[True, False])
    DS['brand_serial'] = DS.groupby('rating_bin').cumcount() + 1
    DS = DS.drop(columns=['product_subcategory'])
    return DS.to_dict(orient='records')



In [48]:
get_heatmap('Hair Care')

Hair Care
7


[{'product_type_count': 6,
  'product_rating_mean': 4.3999999999999995,
  'rating_bin': 1,
  'brand_serial': 1},
 {'product_type_count': 6,
  'product_rating_mean': 4.333333333333333,
  'rating_bin': 1,
  'brand_serial': 2},
 {'product_type_count': 6,
  'product_rating_mean': 4.183333333333334,
  'rating_bin': 1,
  'brand_serial': 3},
 {'product_type_count': 6,
  'product_rating_mean': 3.3833333333333333,
  'rating_bin': 1,
  'brand_serial': 4},
 {'product_type_count': 6,
  'product_rating_mean': 2.75,
  'rating_bin': 1,
  'brand_serial': 5},
 {'product_type_count': 6,
  'product_rating_mean': 2.5833333333333335,
  'rating_bin': 1,
  'brand_serial': 6},
 {'product_type_count': 7,
  'product_rating_mean': 4.328571428571428,
  'rating_bin': 2,
  'brand_serial': 1},
 {'product_type_count': 8,
  'product_rating_mean': 4.2,
  'rating_bin': 2,
  'brand_serial': 2},
 {'product_type_count': 7,
  'product_rating_mean': 4.171428571428572,
  'rating_bin': 2,
  'brand_serial': 3},
 {'product_type_